In [3]:
import torch
from transformers import AutoConfig,AutoModel,AutoTokenizer
from transformers import AutoModelForSequenceClassification,AutoModelForMaskedLM,AutoModelForQuestionAnswering
from transformers import AutoModelForSeq2SeqLM,AutoModelForTokenClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
# 特征提取任务-不带任务输出头的任务
def dm02_test_feature_extraction():
    # 1 加载tokenizer
    my_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=r'D:\bert')

    # 2 加载模型
    my_model = AutoModel.from_pretrained(pretrained_model_name_or_path = r'D:\bert')

    # 3 文本转张量
    message = ['你是谁', '人生该如何起头']
    msgs_tensor = my_tokenizer.encode_plus(text=message, return_tensors='pt', truncation=True, pad_to_max_length=True, max_length=30)
    print('msgs_tensor--->', msgs_tensor)
    #token_type_ids第一个句子和第二个句子  attention_mask中1为句子 后面为补齐
    # 4 给模型送数据提取特征
    my_model.eval()
    output = my_model(**msgs_tensor)#msgs_tensor为字典 传入字典可以直接用**
    print('不带模型头输出output--->', output)
    #最后一个隐藏层的状态
    print('outputs.last_hidden_state.shape--->', output.last_hidden_state.shape)  # torch.Size([1, 30, 768])
    #池化层 防止过拟合 让模型更简单
    print('outputs.pooler_output.shape--->', output.pooler_output.shape)  # torch.Size([1, 768])
dm02_test_feature_extraction()

msgs_tensor---> {'input_ids': tensor([[ 101,  872, 3221, 6443,  102,  782, 4495, 6421, 1963,  862, 6629, 1928,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])}
不带模型头输出output---> BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.7001,  0.4651,  0.2427,  ...,  0.5753, -0.4330,  0.1878],
         [ 0.4017,  0.1123,  0.4482,  ..., -0.2614, -0.2649, -0.1497],
         [ 1.2000, -0.4859,  1.1970,  ...,  0.7543, -0.2404, -0.2627],
         ...,
         [ 0.2074,  0.4022, -0.0448,  ..., -0.0849, -0.0766, -0.2134],
         [ 0.0879,  0.2482, -0.2356,  ...,  0.2967, -0.2357, -0.5138],
         [ 0.4944,  0.1340, -0.2387,  ...,  0.2375, 

In [20]:
# 完型填空任务
def dm03_test_fill_mask():

    # 1 加载tokenizer
    #modename = "chinese-bert-wwm"
    # modename = "bert-base-chinese"
    my_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path =r'D:\bert-wwm')

    # 2 加载模型
    my_model = AutoModelForMaskedLM.from_pretrained(pretrained_model_name_or_path =r'D:\bert-wwm')
    #print(my_model)
    # 3 文本转张量
    input = my_tokenizer.encode_plus('我想明天去[MASK]家吃饭.', return_tensors='pt')#104行为MASK
    print('input--->', input)

    # 4 给模型送数据提取特征
    my_model.eval()
    output = my_model(**input)
    print('output--->', output) 
    print('output.logits--->', output.logits.shape) # [1,12,21128]需要从结果中找到mask对应位输出的21128概率值

    # 5 取概率最高
    mask_pred_idx = torch.argmax(output.logits[0][6]).item()#【6】处即为MASK对应的位置 取这个地方最大的向量值
    print(mask_pred_idx)
    print('打印概率最高的字:', my_tokenizer.convert_ids_to_tokens([mask_pred_idx]))
dm03_test_fill_mask()

Some weights of the model checkpoint at D:\bert-wwm were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


input---> {'input_ids': tensor([[ 101, 2769, 2682, 3209, 1921, 1343,  103, 2157, 1391, 7649,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
output---> MaskedLMOutput(loss=None, logits=tensor([[[ -9.9017,  -9.6006,  -9.8032,  ...,  -7.9744,  -7.7402,  -8.2912],
         [-14.3878, -15.0353, -14.7893,  ..., -10.0437, -10.5279,  -9.7544],
         [-14.2215, -14.1145, -14.5770,  ...,  -6.3246,  -4.1784,  -4.6072],
         ...,
         [-14.6938, -16.8133, -15.1296,  ...,  -9.2327,  -8.1931, -15.2430],
         [-10.8649, -11.4887, -11.5731,  ...,  -6.5378,  -0.8715,  -5.3870],
         [-11.8495, -11.8358, -12.0314,  ...,  -8.4242,  -6.2741,  -8.2787]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)
output.logits---> torch.Size([1, 12, 21128])
1961
打印概率最高的字: ['她']


In [25]:
def dm04_test_question_answering():

    # 1 加载tokenizer
    my_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path =r'D:\roberta')

    # 2 加载模型
    my_model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name_or_path =r'D:\roberta')

    # 3 文本转张量
    # 文字中的标点符号如果是中文的话，会影响到预测结果 也可以去掉标点符号
    context = '我叫张三 我是一个程序员 我的喜好是打篮球'
    questions = ['我是谁？', '我是做什么的？', '我的爱好是什么？']

    # 4 给模型送数据 模型做抽取式问答 一个问题一个问题去解答
    my_model.eval()
    for question in questions:
        input = my_tokenizer.encode_plus(question, context, return_tensors='pt')#前面为问题后面为句子
        print('input--->', input)
        output = my_model(**input)
        print('output--->', output)#从start和end中找到句子起始和结束构成答案
        start, end = torch.argmax(output.start_logits), torch.argmax(output.end_logits) +1
        print('start,end',start,end)
        answer =''.join(my_tokenizer.convert_ids_to_tokens(input['input_ids'][0][start:end] ))#找到对应索引
        print('question:', question, 'answer:', answer)
dm04_test_question_answering()

Some weights of the model checkpoint at D:\roberta were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


input---> {'input_ids': tensor([[ 101, 2769, 3221, 6443, 8043,  102, 2769, 1373, 2476,  676, 2769, 3221,
          671,  702, 4923, 2415, 1447, 2769, 4638, 1599, 1962, 3221, 2802, 5074,
         4413,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
output---> QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -1.9978, -11.4788, -12.6324, -11.8324, -12.4148, -11.9371,  -2.7246,
          -6.6402,   3.9131,  -2.9533,  -7.0866,  -9.5696,  -4.2775,  -8.9042,
           0.5753,  -6.9468,  -7.0469,  -8.5334, -11.3796,  -9.3905, -11.0242,
         -11.1047,  -5.7124,  -2.7293,  -7.5896, -12.6013]],
       grad_fn=<CloneBackward0>), end_logits=tensor([[ -1.3483, -12.0141, -11.6312, -11.6629, -11.9607, -12.0039,  -4.6118,
          -7.4034,  -2.3499,   4.7159,  -7.2880,  -9.5317,  -6.6742,  

In [30]:
# 文本摘要任务
def dm05_test_summarization():
    text = "BERT is a transformers model pretrained on a large corpus of English data " \
           "in a self-supervised fashion. This means it was pretrained on the raw texts " \
           "only, with no humans labelling them in any way (which is why it can use lots " \
           "of publicly available data) with an automatic process to generate inputs and " \
           "labels from those texts. More precisely, it was pretrained with two objectives:Masked " \
           "language modeling (MLM): taking a sentence, the model randomly masks 15% of the " \
           "words in the input then run the entire masked sentence through the model and has " \
           "to predict the masked words. This is different from traditional recurrent neural " \
           "networks (RNNs) that usually see the words one after the other, or from autoregressive " \
           "models like GPT which internally mask the future tokens. It allows the model to learn " \
           "a bidirectional representation of the sentence.Next sentence prediction (NSP): the models" \
           " concatenates two masked sentences as inputs during pretraining. Sometimes they correspond to " \
           "sentences that were next to each other in the original text, sometimes not. The model then " \
           "has to predict if the two sentences were following each other or not."

    # 1 加载tokenizer
    my_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=r"D:\distibart")

    # 2 加载模型
    my_model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path=r'D:\distibart')

    # 3 文本转张量
    input = my_tokenizer([text], return_tensors='pt')
    print('input--->', input)
    my_model.eval()
    output = my_model.generate(input.input_ids)
    print('output--->', output)

    # 5 处理摘要结果
    # 5-1 decode 的 skip_special_tokens 参数可以去除 token 前面的特殊字符
    print(my_tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))

dm05_test_summarization()

input---> {'input_ids': tensor([[    0, 11126,   565,    16,    10,  7891,   268,  1421, 11857, 26492,
            15,    10,   739, 42168,     9,  2370,   414,    11,    10,  1403,
            12, 16101, 25376,  2734,     4,   152,   839,    24,    21, 11857,
         26492,    15,     5,  6087, 14301,   129,     6,    19,   117,  5868,
          6348, 14139,   106,    11,   143,   169,    36,  5488,    16,   596,
            24,    64,   304,  3739,     9,  3271,   577,   414,    43,    19,
            41,  8408,   609,     7,  5368, 16584,     8, 14105,    31,   167,
         14301,     4,   901, 12810,     6,    24,    21, 11857, 26492,    19,
            80, 10366,    35, 47661,   196,  2777, 19039,    36, 10537,   448,
          3256,   602,    10,  3645,     6,     5,  1421, 22422, 17683,   379,
           207,     9,     5,  1617,    11,     5,  8135,   172,   422,     5,
          1445, 24397,  3645,   149,     5,  1421,     8,    34,     7,  7006,
             5, 24397,  1617

In [32]:
#NER任务
def dm06_test_ner():
    # 1 加载tokenizer 加载模型 加载配置文件
    # https://huggingface.co/uer/roberta-base-finetuned-cluener2020-chinese
    my_tokenizer = AutoTokenizer.from_pretrained(r'D:\cnn')
    my_model = AutoModelForTokenClassification.from_pretrained(r'D:\cnn')
    config = AutoConfig.from_pretrained(r'D:\cnn')
    # 2 数据张量化
    inputs = my_tokenizer.encode_plus('鲁迅先生的代表作是《狂人日记》', return_tensors='pt')
    print('inputs--->', inputs.input_ids.shape, inputs.input_ids) # torch.Size([1, 17])

    # 3 送入模型 预测ner概率 每个字预测的标签概率
    my_model.eval()
    logits = my_model(inputs.input_ids).logits
    print('logits--->', logits.shape)           # torch.Size([1, 17, 32])

    # 4 对预测数据 进行显示
    input_tokens = my_tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
    print('input_tokens--->', input_tokens)
    outputs = []

    for token, value in zip(input_tokens, logits[0]):

        if token in my_tokenizer.all_special_tokens:#去除[CLS][SEP]
            continue

        # 获得每个字预测概率最大的标签索引
        idx = torch.argmax(value).item()

        # 打印索引对应标签
        outputs.append((token, config.id2label[idx]))

    print(outputs)
dm06_test_ner()

inputs---> torch.Size([1, 17]) tensor([[ 101, 7826, 6813, 1044, 4495, 4638,  807, 6134,  868, 3221,  517, 4312,
          782, 3189, 6381,  518,  102]])
logits---> torch.Size([1, 17, 32])
input_tokens---> ['[CLS]', '鲁', '迅', '先', '生', '的', '代', '表', '作', '是', '《', '狂', '人', '日', '记', '》', '[SEP]']
[('鲁', 'B-name'), ('迅', 'I-name'), ('先', 'O'), ('生', 'O'), ('的', 'O'), ('代', 'O'), ('表', 'O'), ('作', 'O'), ('是', 'O'), ('《', 'B-book'), ('狂', 'I-book'), ('人', 'I-book'), ('日', 'I-book'), ('记', 'I-book'), ('》', 'I-book')]


In [1]:
from transformers import BertTokenizer,BertForMaskedLM

C:\Users\Lenovo\.conda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Disabling PyTorch because PyTorch >= 2.1 is required but found 1.12.0+cu113


In [36]:
#具体模型方式完成完形填空任务
# 完型填空任务
def dm03_test_fill_mask():

    # 1 加载tokenizer
    #modename = "chinese-bert-wwm"
    # modename = "bert-base-chinese"
    my_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path =r'D:\bert-wwm')

    # 2 加载模型
    my_model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path =r'D:\bert-wwm')#和自动模型区别只在于导包不相同
    #print(my_model)
    # 3 文本转张量
    input = my_tokenizer.encode_plus('我想明天去[MASK]家吃饭.', return_tensors='pt')#104行为MASK
    print('input--->', input)

    # 4 给模型送数据提取特征
    my_model.eval()
    output = my_model(**input)
    print('output--->', output) 
    print('output.logits--->', output.logits.shape) # [1,12,21128]需要从结果中找到mask对应位输出的21128概率值

    # 5 取概率最高
    mask_pred_idx = torch.argmax(output.logits[0][6]).item()#【6】处即为MASK对应的位置 取这个地方最大的向量值
    print(mask_pred_idx)
    print('打印概率最高的字:', my_tokenizer.convert_ids_to_tokens([mask_pred_idx]))
dm03_test_fill_mask()

Some weights of the model checkpoint at D:\bert-wwm were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


input---> {'input_ids': tensor([[ 101, 2769, 2682, 3209, 1921, 1343,  103, 2157, 1391, 7649,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
output---> MaskedLMOutput(loss=None, logits=tensor([[[ -9.9017,  -9.6006,  -9.8032,  ...,  -7.9744,  -7.7402,  -8.2912],
         [-14.3878, -15.0353, -14.7893,  ..., -10.0437, -10.5279,  -9.7544],
         [-14.2215, -14.1145, -14.5770,  ...,  -6.3246,  -4.1784,  -4.6072],
         ...,
         [-14.6938, -16.8133, -15.1296,  ...,  -9.2327,  -8.1931, -15.2430],
         [-10.8649, -11.4887, -11.5731,  ...,  -6.5378,  -0.8715,  -5.3870],
         [-11.8495, -11.8358, -12.0314,  ...,  -8.4242,  -6.2741,  -8.2787]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)
output.logits---> torch.Size([1, 12, 21128])
1961
打印概率最高的字: ['她']
